In [1]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'calibrate.ipynb'

# IonPy imports
from ionpy.util import Config

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [2]:
%%yaml default_cfg

experiment:
  restart: True # Important, otherwise we don't load the model.

train:
  epochs: '?' # How much longer to train for.
  load_chkpt: '?' # Which model do we load
  pretrained_dir: '?' # Which runs are we restarting

<IPython.core.display.Javascript object>

In [3]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.callbacks.ShowPredictions
  epoch:
    - ese.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

<IPython.core.display.Javascript object>

In [4]:
%%yaml experiment_cfg 

name: "ISLES_3D_Dice_wA_MixedPrec_lowLR_RESTARTED" # We will treat this as a NEW EXPERIMENT.

train:
    epochs: 1000
    load_chkpt: 'last'
    pretrained_dir: "/storage/vbutoi/scratch/ESE/training/09_18_24_ISLES_3D_Dice_wA_MixedPrec_lowLR"

<IPython.core.display.Javascript object>

In [5]:
from ese.analysis.analysis_utils.submit_utils import get_ese_restart_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([callbacks_cfg])

updated_base_cfg, restart_cfgs = get_ese_restart_configs(
    exp_cfg=experiment_cfg,
    base_cfg=base_cfg
)

In [6]:
len(restart_cfgs)

3

## Running Jobs

In [7]:
from ese.experiment import run_ese_exp, submit_ese_exps, CalibrationExperiment, PostHocExperiment

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [8]:
# ####### Run individual jobs
# run_ese_exp(
#     config=restart_cfgs[0], 
#     experiment_class=CalibrationExperiment,
#     run_name='debug',
#     show_examples=True,
#     gpu='0'
# )

In [9]:
#### Run Batch Jobs
submit_ese_exps(
    group="restart",
    base_cfg=updated_base_cfg,
    exp_cfg=experiment_cfg,
    config_list=restart_cfgs,
    experiment_class=CalibrationExperiment,
    track_wandb=True,
    available_gpus=['5', '6', '7']
)

Submitted job id: 2869518 on gpu: 5.
Submitted job id: 2869832 on gpu: 6.
Submitted job id: 2870059 on gpu: 7.
